# Backtest 2020

## What is the backtest?

The backtest analyzes data from a past period of time to determine the ideal values of seven different metrics (discussed later) for each sector.  This version of the backtest uses the jupyter notebook format to describe each part, as well as to increase efficiency.

###  
### Library Imports

- pandas - used to manipulate data within different data structures
- datetime - used to get today's date and current time
- time - used for calculating runtime
- numpy - supports mathematical functions for large volume data structures
- product - creates a cartesian product

In [37]:
from pandas import *
import pandas as pd
from datetime import *
import time
import numpy as np
from itertools import product
from IPython.display import display

### Effiency and timing

In the past, it has taken dozens of hours to complete the backtests.  This is costly in terms of time and electricity (which should be considered as a sustainability-oriented organization).  The block below assigns the current time to a variable so that we can later determine runtime.

We also establish our sectors and metrics in lists.

In [38]:
metrics = [ 'PE_RATIO', 'PX_TO_BOOK_RATIO', 'TRAIL_12M_EPS', 'TOT_DEBT_TO_TOT_EQY', \
               'PX_TO_FREE_CASH_FLOW', 'RETURN_COM_EQY', 'RETURN_ON_ASSET' ]

#These are used in creating and manipulating dataframes
dfTests = ['peTest','pbTest','epsTest','deTest','fcfTest','roeTest','roaTest']
dfScores = ['peScore','pbScore','epsScore','deScore','fcfScore','roeScore','roaScore']

lvhs_metrics = [ 'PE_RATIO', 'TOT_DEBT_TO_TOT_EQY', 'PX_TO_BOOK_RATIO' ] # Low-value high score metrics
hvhs_metrics = [ 'TRAIL_12M_EPS', 'PRICE_TO_FCF', 'RETURN_COM_EQY', 'RETURN_ON_ASSET' ] # High value high score metrics

pd.options.mode.chained_assignment = None #allow chained assignment

#  
#   
# Step 1: Data input and Setup, Organic Range Generation

The basis of our backtest are frames - sets of each metric, each containing a different combination of values.  In order to determine what these values are, we have to come up with a range for each metric, for each sector.

### Data Import

We first import our data file for range generation -- the last 10 years of data for each company.  The data is imported to a **dataframe**, a data structure which can be thought of as an excel sheet manipulable through python.  The dataframe is then sorted by date.

The line reading "sdf.dropna..." drops rows where the date is empty.  We then establish a start date (generally five years prior to the test being run), and establish the date format as "ordinal."  This ensures that our date formatting is compatible with that of our code.  Finally, we cut the data based on when our start date is, so that we are not using more data than is necessary and doing an excessive amount of computing.

In [39]:
sdf = read_csv("Backtest VALUES 2019.csv",index_col='DATE',parse_dates=True)

sdf = sdf.sort_index()

#drop any rows without a date
sdf.dropna(axis='index',how='any',inplace=True)

#Define starting date to begin df with
startDate = to_datetime('2014-03-28')
startDate=startDate.toordinal()

#Cut spyx_Sector based on DateTime range
sdf['ordinalTime']=sdf.index
sdf['ordinalTime'] = sdf['ordinalTime'].apply(date.toordinal)
sdf = sdf.loc[sdf['ordinalTime']>=startDate]

#Create set of all of the dates contained in sdf
datesList = sorted(set(sdf.index))
sectors = sorted(set(sdf.GICS_SECTOR_NAME.unique()))

### getRanges method

The getRanges method returns a list of ranges, one for each metric, for that specific sector.  A range has a low value, high value, and step value (incrementation value).  While this code is visually unappealing, it's relatively practical. 

It is worth noting that we take ranges from the 25th percentile to the 75th percentile (the middle section of a normal distribution.  If you're unsure what a normal distribution looks like and why we chose these percentages, I suggest researching these so that you have a better understanding of the statistics behind this).  The companies in the top 25% for a metric may be outliers or overvalued based on that number, and the companies in the lower 25% may be underperforming.  

It is also worth noting the step values.  These are generally safe bets for typical output to create a reasonable number of frames.  Remember, runtime is **_exponential_**.*  One of our current tasks for improving the model is to develop an algorithm to calculate the step value; for example if the range starts at 20 and goes up to 120, a step of 10 has 12 potential frame values, while a step value of 5 has 24.  The fewer frame values we have, the more quickly the code will run; too large of a step value, and we may be distorting the accuracy of the backtest.

First, we have to import the data again as an excel sheet.  This is due to some conflicts with "None" values as read by a CSV file.  We then sort the data by date and group the rows by sector.

We then define a function which gets the appropriate data for each metric and returns the ranges as a tuple.

In [50]:
#Using the same CSV file creates issues with the slice(None) function
#Try to fix this .xlsx issue wher it doesn't allow slicing
sdf2 = pd.read_excel("Backtest VALUES 2019.xlsx", index_col=[0,1])
sdf2 = sdf2.sort_index()
sdf2.groupby('GICS_SECTOR_NAME').nunique()

#Unlike typical range functions, the step value must be listed, even if it is 1
#This function calculates and returns the normal range given the sector, metric, and step
def getNormal(metric, step):
    return range(int(round(np.percentile(sdf2.loc[(slice(None), sector), metric].dropna(), 25))), \
                     int(round(np.percentile(sdf2.loc[(slice(None), sector), metric].dropna(), 75))), step)

def getRanges():
    #define these values as global so that we can use them in other functions
    global peRange, pbRange, epsRange, deRange, fcfRange, roeRange, roaRange
    
    peRange =  getNormal('PE_RATIO', 2)
    pbRange =  getNormal('PX_TO_BOOK_RATIO', 1)
    epsRange =  getNormal('TRAIL_12M_EPS', 2)
    deRange =  getNormal('TOT_DEBT_TO_TOT_EQY', 10)
    fcfRange =  getNormal('TOT_DEBT_TO_TOT_EQY', 5)
    roeRange =  getNormal('RETURN_COM_EQY', 2)
    roaRange =  getNormal('RETURN_ON_ASSET', 2)

#  
#  
# Step 2: Function Definitions

### Output functions

The backtest prints out three sheets as output.

SectorNameIndPortPeriodDate is a sheet showing hypothetical groupings of companies, per quarter, which fit each frame.  It gives the data for each of these companies during that quarter.  This is the largest sheet; the number of frames times the number of quarters = the number of rows in this spreadsheet. 

SectorNameIndividualPortfoliosDate takes the same data and displays it without showing each individual company in each test.  It shows the results of how each frame performed each quarter.

SectorNameTotalPortfolioDate prints out the ultimate performance of each frame's average performance over time.  This is ordered by Treynor ratio, so the best frame (highest Treynor ratio) will be the first line.

In [51]:
#this should fix the flipping dates issue
now = time.strftime("d%dm%my%Y")
#Function to output every indivdual portfolio inside each period
def outputIndPeriod(df, date):
        df.to_csv(sector+"IndPortPeriod"+now+".csv", mode='a')
        
#output each portfolio with dates
def outputIndPort(dfPort, params):
    dfPort.to_csv(sector+"IndividualPortfolios"+now+".csv", \
                  mode='a',index_label=("T"+str(params[0])+str(params[1])+str(params[2])+str(params[3])\
                        +str(params[4])+str(params[5])+str(params[6])))

#Output total portfolio with parameters
def outputTotalPort(dfTotalPort, params):
    dfTotalPort.to_csv(sector+"TotalPortfolio"+now+".csv", mode='w',index_label=("T"+"Portfolio"))
    

### Calculate Portfolios

To understand this function, it's also important to understand the Cartesian product and the prod function.

Let's look at the function definition.  The datesList comes from way down below; we basically create a list of all of the dates in our data.  "Params" comes from the prod function; it contains a frame, which is a list containing a hypothetical value for each metric.  Sect is simply the name of the sector.  Params, or the frame values, only changes when the calculatePortLoop is called from the loop in the prod() function.

The for loop is based around our dates list.  For each date (which equates each quarter), we want to do the following:

1. Take all data pertaining to that date/quarter and create a new dataframe (df.
2. Format the dataframe based on our metrics and insert the frame values.
3. Score whether or not the actual data is at least as good as the frame value.  This is listed as a TRUE or FALSE value.
4. Calculate the HSF score. This counts the number of TRUE values from the last step and adds the adjusted beta, which has been divided by 1000 so as not to disrupt the innate value of the HSF score.
5. An identifying tag is established for each frame.  The frame value is used to generate this tag.  This is sort of like an SKU code used in retail.
6. The dataframe is sorted by highest HSF Score and the top 5 companies are selected.  These are added to the SectorNameIndPortPeriodDate.csv file.


7. The tested frame values are added to the dfPort dataframe, which is established much further below.  The highest HSF score, as well as the average return and average adjusted beta, are also added to this dataframe.  
8. The average return and beta are then used to calculate a Treynor ratio; basically, this represents risk vs. return for those top 5 HSF scoring companies for the frame and date. 
9. The frame is also added, formatted as a list like (0, 1, 2, etc).  dfPort is then returned (to the prod() function).

In [52]:
def createIdentifier(params):
    name = "T"
    for p in params:
        name+=str(p)
    return name

In [53]:
#Function to loop through every date and output an individual portfolio
def calculatePortLoop(dateList, params, identity):
    
    #Go through the dates list.  Params will remain the same through the for loop, and only changes when this function
    #is called again from the prod() function.
    for date in datesList:
        
        #create a dataframe based on the date
        df = spyx_Sector.loc[spyx_Sector.index==date]
        
        #Adds the values of the frame to the dataframe
        for x in range(7):
            df[dfTests[x]] = params[x]
        
        #Scores whether or not the value for each metric is at least as good as the frame value
        for x in range(7):
            if metrics[x] in lvhs_metrics:
                df[dfScores[x]] = df[metrics[x]] < params[x]
            else:
                df[dfScores[x]] = df[metrics[x]] > params[x]
        
        #Re-calculate beta score so that we can add it to the HSF score
        df['betaScore']=1/(df['ADJUSTED_BETA']*1000)
        #Calculate HSF score
        df['hsfScore']=df['peScore'].astype(int)+df['pbScore'].astype(int)+df['epsScore'].astype(int)+\
            df['deScore'].astype(int)+df['fcfScore'].astype(int)+df['roeScore'].astype(int)+df['roaScore'].astype(int)
        #Creates an identifier for each frame based on the frame values
        df['ind'] = identity
        #"T"+df['peTest'].map(str)+df['pbTest'].map(str)+df['epsTest'].map(str)+df['deTest'].map(str)+\
            #df['fcfTest'].map(str)+df['roeTest'].map(str)+df['roaTest'].map(str)
        
        #Sorts the dataframe by highest HSF score
        df.sort_values(by='hsfScore',ascending=False,inplace=True)
        #Add only the 5 companies with the highest HSF score to the dataframe.
        df=df.iloc[0:5]
        
        #Print to the SectorNameIndPortPeriodDate.csv file.  This happens each time we go through the loop
        if iperiods:
            outputIndPeriod(df, date)
        
        #DFPort is for SectorNameIndividualPortfoliosDate.csv
        for t in dfTests:
            dfPort.loc[dfPort.index==date, t] = df.ix[0, t]
        
        dfPort.loc[dfPort.index==date,'maxHSFScore'] = df['hsfScore'].max()
        #Finds the average return and adjusted beta of the five companies.
        dfPort.loc[dfPort.index==date,'return'] = (df['RETURN'].mean())
        #this is where the float error comes into play
        dfPort.loc[dfPort.index==date,'beta'] = (df['ADJUSTED_BETA'].mean()+0.0000001)

        
    #Calculate a Treynor ratio for the dfPort dataframe
    dfPort['treynor']=(dfPort['return']-TBill3Mth)/dfPort['beta']
    #Adds the frame in list formatting to the dfPort dataframe
    dfPort['ind'] = identity
    #="T"+dfPort['peTest'].map(str)+dfPort['pbTest'].map(str)+dfPort['epsTest'].map(str)+dfPort['deTest'].map(str)+\
        #dfPort['fcfTest'].map(str)+dfPort['roeTest'].map(str)+dfPort['roaTest'].map(str)
    return dfPort

### Cartesian Product

Now, we'll look at the Cartesian product.  This is where things get slightly more mathematical, and where we can fully grasp the importance of efficiency.  

The only argument of the prod() function is the sector name, and this is just to pass to other functions which need that name to create our .csv files.

This for loop is the heart of our backtest.  Keep in mind that in python, a function can be called in the for loop if it returns a list which can be traversed.  In this case, that function is the product() function; more specifically, this creates a Cartesian product.  From thefreedictionary.com:

    A cartesian product is "the set of all ordered pairs of members of two given sets. The product A × B is the set of all pairs <a, b> where a is a member of A and b is a member of B."
    
Think of it like this.  Say we have two sets of numbers, A and B.  Say that A has the values (1, 2, 3) and B has the values (4, 5, 6).  A cartesian product between A and B would produce  all of these possible sets:

(1, 4)
(1, 5)
(1, 6)
(2, 4)
(2, 5)
(2, 6)
(3, 4)
(3, 5)
(3, 6)

Each number from A is combined with each number from B.  Note that the number of possible sets is equal to the number of values in A times the number of values in B: 3 x 3 = 9.  If B had four numbers, there would be 3 x 4 = 12 possible combinations, and so forth and so on.  Now consider how this works with our ranges; an example for the number of values in each range is below:

    peRange = 4 possible values

    pbRange = 5 possible values

    epsRange = 6 possible values

    deRange = 4 **

    fcfRange = 4 **

    roeRange = 4 **

    roaRange = 3 **


4 x 5 x 6 x 4 x 4 x 4 x 3 = 23,040 possible combinations.  This is the number of frames that the for loop would have to run through.  Say that roaRange changed from 3 to 4; this would give us 30,720 combinations.  As you can see, the ranges create an exponentially large set of combinations; this is why the backtest takes so long to run, and why efficiency is so important.

So how does the rest of the function work?  

1. The for loop takes a single frame from the cartesian product and calls the calculatePortLoop function from above.  As we know, this adds data to the SectorNameIndPortPeriodDate.csv file and to the dfPort dataframe.
2. dfTotalPort is the dataframe to show the final results.  We calculate the average return for the frame over time as well as the average beta, and insert these into the dfTotalPort dataframe.
3. The frame values are copied into dfTotalPort.
4. The SectorNameIndividualPortfoliosDate.csv is printed to using the dfPort dataframe.
5. The loop continues to do this for each frame.
6. After the loop has finished, the total Treynor for each frame is calculated using the average beta and average return for each frame over time, as well as risk-free rate.  This data is added to the dfTotalPort dataframe.
7. The dfTotalPort dataframe is then sorted by Treynor ratio, with the highest Treynors appearing at the top. Keep in mind that Treynor ratio shows return vs. risk.  Higher Treynor values are considered better, so the first Treynor listed on this sheet will be for the best frame.
8. Similar to above, identifying tags are again generated for each frame.  They match the format as in the last function.
9.  outputTotalPort is called, allowing for us to print out the SectorNameTotalPortfolioDate.csv file.

In [54]:
def prod():
    
    #df, dfport, dfTotalPort
    for params in product(peRange, pbRange, epsRange, deRange, fcfRange, roeRange, roaRange):
        identifier = createIdentifier(params)
        #Calculate portfolios
        calculatePortLoop(datesList, params, identifier)
        #Average return for the frame over time
        dfTotalPort.loc[dfTotalPort['ind']==params,'meanReturn'] = dfPort['return'].mean()
        #Average Beta for the frame over time
        dfTotalPort.loc[dfTotalPort['ind']==params,'meanBeta'] = dfPort['beta'].mean()
        #Repeats the frame values into the dataframe
        for x in range(7):
            dfTotalPort.loc[dfTotalPort['ind']==params, dfTests[x]] = params[x]
        
        #print(dfPort)
        if iports:
            outputIndPort(dfPort, params)
        
    #Calculate Treynor
    dfTotalPort['totalTreynor']=(dfTotalPort['meanReturn']-dfTotalPort['tBill3Mth'])/dfTotalPort['meanBeta']
    #Sort by Treynor
    dfTotalPort.sort_values('totalTreynor',ascending=False, inplace=True)
    #Creates an identifying string using the values of the frame
    #ammend this dfTotalPort to be able to use our function instead
    dfTotalPort.set_index("T"+dfTotalPort['peTest'].map(str) + dfTotalPort['pbTest'].map(str) +\
                      dfTotalPort['epsTest'].map(str) + dfTotalPort['deTest'].map(str) +\
                      dfTotalPort['fcfTest'].map(str) + dfTotalPort['roeTest'].map(str) +\
                      dfTotalPort['roaTest'].map(str), inplace = True)
    
    #Print to CSV file
    outputTotalPort(dfTotalPort, params)

#  
#  
# Step 3: Naming sector, Producing Output Per Sector

There are two functions below.  The first runs the backtest for a single sector.  The second runs the backtest for every sector.

We first name a sector.  The ranges for this sector are then generated by calling the getRanges(sector) function, which returns a tuple containing each range.

A new dataframe, spyx_Sector, is created.  It takes all of the data from sdf that pertains to companies in the defined sector.

The output dataframes are initialized as empty.  dfPort will be used for the SectorNameIndividualPortfoliosDate.csv file, and dfTotalPort will be used for the SectorNameTotalPortfolioDate.csv file, as explained above.

Finally, the prod() function is called.  This will run the Cartesian product and all of the output functions, printing all of the necessary data to our csv files.

In [57]:
def runSingleSector(sectorChosen, tbill, libor, IndPortfolios, IndPeriods):

    global sector
    sector = sectorChosen
    
    getRanges()

    #define spyx_Sector for use in other functions
    global spyx_Sector
    spyx_Sector = sdf.loc[sdf['GICS_SECTOR_NAME']==sector]
    
    #define the risk-free rates
    global TBill3Mth
    TBill3Mth = tbill/100
    global Libor3Mth
    Libor3Mth = libor/100
    
    #define whether or not we want excel sheets containing individual portfolio models and periodic models
    global iports
    iports = IndPortfolios
    global iperiods
    iperiods = IndPeriods
    

    #Create new Output DF for portfolios for dates
    global dfPort
    dfPort = pd.DataFrame([],index=datesList)
    dfPort['return']=None
    dfPort['beta']=None
    dfPort['tBill3Mth']=TBill3Mth
    dfPort['treynor']=None
    for x in range(7):
            dfPort[dfTests[x]] = None
    dfPort['maxHSFScore']=None

    #Create new Output DF for total portfolios
    global dfTotalPort
    dfTotalPort = pd.DataFrame([])
    dfTotalPort['ind']=list(product(peRange,pbRange,epsRange,deRange,fcfRange,roeRange,roaRange))
    dfTotalPort['meanReturn']=None
    dfTotalPort['meanBeta']=None
    dfTotalPort['tBill3Mth']=TBill3Mth
    dfTotalPort['totalTreynor']=None
    for x in range(7):
            dfTotalPort[dfTests[x]] = None

    #call the prod() function
    prod()

#  
#  
# Function Calls and Runtime

The following calls the desired function and calculates runtime.  In order to call the function, we have to put in our Risk-Free rates (T-Bill 3-month average and Libor 3-month average) in order to run the function.  These change regularly, so they should be checked each time the model is run.

In [58]:
start = time.time()

runSingleSector('Telecommunication Services', 1.52, 1.81, True, True)

#for s in sectors:
#    runSingleSector(s, tbill, libor, False, False)

stop = time.time()

print('Time: ', stop - start)  

C:\Users\2rtma\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


KeyboardInterrupt: 